## 导包

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #默认为'last'
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import sys
import random
from numpy import linalg
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)

# torch.sparse

In [11]:
i = torch.LongTensor([[0, 1, 1], [2, 0, 2]])
v = torch.FloatTensor([3, 4, 5])
torch.sparse.FloatTensor(i, v, torch.Size([2,3])).to_dense()

tensor([[0., 0., 3.],
        [4., 0., 5.]])

In [5]:
i = torch.LongTensor([[0, 2], [1, 0], [1, 2]])
v = torch.FloatTensor([3, 4, 5])
torch.sparse.FloatTensor(i.t(), v, torch.Size([2, 3])).to_dense()

tensor([[0., 0., 3.],
        [4., 0., 5.]])

In [6]:
i = torch.LongTensor([[2, 4]])
i
v = torch.FloatTensor([[1, 3], [5, 7]])
v
torch.sparse.FloatTensor(i, v)
torch.sparse.FloatTensor(i, v).to_dense()

tensor([[2, 4]])

tensor([[1., 3.],
        [5., 7.]])

tensor(indices=tensor([[2, 4]]),
       values=tensor([[1., 3.],
                      [5., 7.]]),
       size=(5, 2), nnz=2, layout=torch.sparse_coo)

tensor([[0., 0.],
        [0., 0.],
        [1., 3.],
        [0., 0.],
        [5., 7.]])

In [7]:
a = torch.randn(2, 3).to_sparse().requires_grad_(True)
a
a.to_dense()
# tensor(indices=tensor([[0, 0, 0, 1, 1, 1],
#                        [0, 1, 2, 0, 1, 2]]),
#        values=tensor([ 1.5901,  0.0183, -0.6146,  1.8061, -0.0112,  0.6302]),
#        size=(2, 3), nnz=6, layout=torch.sparse_coo, requires_grad=True)

b = torch.randn(3, 2, requires_grad=True)
b
# tensor([[-0.6479,  0.7874],
#         [-1.2056,  0.5641],
#         [-1.1716, -0.9923]], requires_grad=True)

y = torch.sparse.mm(a, b)
y
# tensor([[-0.3323,  1.8723],
#         [-1.8951,  0.7904]], grad_fn=<SparseAddmmBackward>)
y.sum().backward()
a.grad
# tensor(indices=tensor([[0, 0, 0, 1, 1, 1],
#                        [0, 1, 2, 0, 1, 2]]),
#        values=tensor([ 0.1394, -0.6415, -2.1639,  0.1394, -0.6415, -2.1639]),
#        size=(2, 3), nnz=6, layout=torch.sparse_coo)


tensor(indices=tensor([[0, 0, 0, 1, 1, 1],
                       [0, 1, 2, 0, 1, 2]]),
       values=tensor([ 2.0876, -0.3764,  0.4160, -0.5974, -1.8790,  1.1513]),
       size=(2, 3), nnz=6, layout=torch.sparse_coo, requires_grad=True)

tensor([[ 2.0876, -0.3764,  0.4160],
        [-0.5974, -1.8790,  1.1513]], grad_fn=<ToDenseBackward>)

tensor([[ 2.1082,  0.7027],
        [ 1.6664,  0.8173],
        [-1.6953,  0.5092]], requires_grad=True)

tensor([[ 3.0686,  1.3711],
        [-6.3424, -1.3692]], grad_fn=<SparseAddmmBackward>)

tensor(indices=tensor([[0, 0, 0, 1, 1, 1],
                       [0, 1, 2, 0, 1, 2]]),
       values=tensor([ 2.8109,  2.4837, -1.1860,  2.8109,  2.4837, -1.1860]),
       size=(2, 3), nnz=6, layout=torch.sparse_coo)

# einsum

In [2]:
x = torch.randn(5)
y = torch.randn(4)
torch.einsum('i,j->ij', x, y)  # outer product
# tensor([[-0.0570, -0.0286, -0.0231,  0.0197],
#         [ 1.2616,  0.6335,  0.5113, -0.4351],
#         [ 1.4452,  0.7257,  0.5857, -0.4984],
#         [-0.4647, -0.2333, -0.1883,  0.1603],
#         [-1.1130, -0.5588, -0.4510,  0.3838]])


A = torch.randn(3,5,4)
l = torch.randn(2,5)
r = torch.randn(2,4)
torch.einsum('bn,anm,bm->ba', l, A, r) # compare torch.nn.functional.bilinear
# tensor([[-0.3430, -5.2405,  0.4494],
#         [ 0.3311,  5.5201, -3.0356]])


As = torch.randn(3,2,5)
Bs = torch.randn(3,5,4)
torch.einsum('bij,bjk->bik', As, Bs) # batch matrix multiplication
# tensor([[[-1.0564, -1.5904,  3.2023,  3.1271],
#          [-1.6706, -0.8097, -0.8025, -2.1183]],

#         [[ 4.2239,  0.3107, -0.5756, -0.2354],
#          [-1.4558, -0.3460,  1.5087, -0.8530]],

#         [[ 2.8153,  1.8787, -4.3839, -1.2112],
#          [ 0.3728, -2.1131,  0.0921,  0.8305]]])

A = torch.randn(3, 3)
torch.einsum('ii->i', A) # diagonal
# tensor([-0.7825,  0.8291, -0.1936])

A = torch.randn(4, 3, 3)
torch.einsum('...ii->...i', A) # batch diagonal
# tensor([[-1.0864,  0.7292,  0.0569],
#         [-0.9725, -1.0270,  0.6493],
#         [ 0.5832, -1.1716, -1.5084],
#         [ 0.4041, -1.1690,  0.8570]])

A = torch.randn(2, 3, 4, 5)
torch.einsum('...ij->...ji', A).shape # batch permute
# torch.Size([2, 3, 5, 4])

tensor([[ 1.3010,  0.1340, -2.2113, -0.6608],
        [ 0.7092,  0.0731, -1.2054, -0.3602],
        [-0.5546, -0.0571,  0.9426,  0.2817],
        [ 0.6613,  0.0681, -1.1240, -0.3359],
        [ 0.6079,  0.0626, -1.0333, -0.3088]])

tensor([[-0.6743,  0.2117, -0.1060],
        [-5.9914,  2.7196, -8.2246]])

tensor([[[-0.9479, -0.8942,  0.6105,  1.2177],
         [ 5.1920, -2.0442,  3.7597,  0.2774]],

        [[ 1.9755,  3.0665,  0.5966, -2.7625],
         [-1.7776, -4.3762, -0.1269, -1.9066]],

        [[ 1.1460,  1.0215, -0.8949,  1.3821],
         [-0.1486, -1.9964,  1.4362, -0.9691]]])

tensor([-0.5741,  0.7686, -1.5141])

tensor([[-0.9746, -1.4177, -1.3442],
        [-0.2116,  0.5845,  0.7181],
        [ 0.2219, -0.7531,  1.5207],
        [-0.9117, -1.1787,  0.6000]])

torch.Size([2, 3, 5, 4])

In [38]:
a = torch.arange(24).reshape(2,3,4)
b = torch.arange(12).reshape(3,2,2)
b = torch.ones(3,2,2)
a
b
# torch.einsum('bij,bjk', a, b）
c = torch.einsum('ij...,jkl->ijkl', a, b)
c
c.shape

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]])

tensor([[[[ 6.,  6.],
          [ 6.,  6.]],

         [[22., 22.],
          [22., 22.]],

         [[38., 38.],
          [38., 38.]]],


        [[[54., 54.],
          [54., 54.]],

         [[70., 70.],
          [70., 70.]],

         [[86., 86.],
          [86., 86.]]]])

torch.Size([2, 3, 2, 2])

In [59]:
a = torch.arange(24.).reshape(2,2,3,2)
# b = torch.arange(12).reshape(3,2,2)
b = torch.Tensor([1,2]).repeat(2, 3, 2)
a
b
c = torch.einsum('bij...,bjk->bijk', a, b)
# c = torch.einsum('ij,jk->ik', a, b)
c
c.shape

tensor([[[[ 0.,  1.],
          [ 2.,  3.],
          [ 4.,  5.]],

         [[ 6.,  7.],
          [ 8.,  9.],
          [10., 11.]]],


        [[[12., 13.],
          [14., 15.],
          [16., 17.]],

         [[18., 19.],
          [20., 21.],
          [22., 23.]]]])

tensor([[[1., 2., 1., 2.],
         [1., 2., 1., 2.],
         [1., 2., 1., 2.]],

        [[1., 2., 1., 2.],
         [1., 2., 1., 2.],
         [1., 2., 1., 2.]]])

tensor([[[[ 1.,  2.,  1.,  2.],
          [ 5., 10.,  5., 10.],
          [ 9., 18.,  9., 18.]],

         [[13., 26., 13., 26.],
          [17., 34., 17., 34.],
          [21., 42., 21., 42.]]],


        [[[25., 50., 25., 50.],
          [29., 58., 29., 58.],
          [33., 66., 33., 66.]],

         [[37., 74., 37., 74.],
          [41., 82., 41., 82.],
          [45., 90., 45., 90.]]]])

torch.Size([2, 2, 3, 4])

In [44]:
help(a.repeat)

Help on built-in function repeat:

repeat(...) method of torch.Tensor instance
    repeat(*sizes) -> Tensor
    
    Repeats this tensor along the specified dimensions.
    
    Unlike :meth:`~Tensor.expand`, this function copies the tensor's data.
    
    .. warning::
    
        :func:`torch.repeat` behaves differently from
        `numpy.repeat <https://docs.scipy.org/doc/numpy/reference/generated/numpy.repeat.html>`_,
        but is more similar to
        `numpy.tile <https://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html>`_.
        For the operator similar to `numpy.repeat`, see :func:`torch.repeat_interleave`.
    
    Args:
        sizes (torch.Size or int...): The number of times to repeat this tensor along each
            dimension
    
    Example::
    
        >>> x = torch.tensor([1, 2, 3])
        >>> x.repeat(4, 2)
        tensor([[ 1,  2,  3,  1,  2,  3],
                [ 1,  2,  3,  1,  2,  3],
                [ 1,  2,  3,  1,  2,  3],
             

# end